In [2]:
import os
import numpy as np
from scipy.stats import multivariate_normal

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
def random_cov_matrix(n=2):
    A = np.random.rand(n, n)
    return A @ A.T

In [5]:
def calculate_multivariate_normal_distribution(dataset, mean, cov, dimensions):
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)

    norm_factor = np.sqrt(((2 * np.pi) ** dimensions) * det_cov)
    norm_factor = 1 / norm_factor
    diff = (dataset - mean)

    power = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)

    result = norm_factor * np.exp(power)
    return result

In [32]:
file_path = os.environ.get("TWO_GAUSSIAN_DATASET_HW2A_QUESTION3")
file = file_path + "2gaussian_hw2A_q3.txt"

dataset = np.loadtxt(file)
number_of_points, dimensions = dataset.shape
print("number of points", number_of_points)
print("dimensions", dimensions)
print("Top 5 entries:\n", dataset[:5])

k = 2

# np.random.seed(96)
# means = np.random.choice(6000, size=2, replace=False)
# means = dataset[means]
means = np.random.random((2,2))
covariance = np.array([random_cov_matrix() for _ in range(k)])  #TODO use genric covariance randomly generated.
weights = np.random.dirichlet([2, 4])# use random weights

print("Initial means:", means)
print("Initial covariance:", covariance, covariance.shape)
print("weights", weights)


threshold = 1e-8
max_iter = 100
prev_likelihood = -np.inf

for iteration in range(max_iter):
    responsibilities = np.zeros((number_of_points, k))

    for ki in range(k):
        # responsibilities[:, ki] = weights[ki] * multivariate_normal.pdf(dataset, means[ki], covariance[ki]) #TODO write your own func for pdf
        responsibilities[:, ki] = weights[ki] * calculate_multivariate_normal_distribution(dataset, means[ki], covariance[ki], k)

    responsibilities /= responsibilities.sum(axis=1, keepdims=True)

    Nk = responsibilities.sum(axis=0)
    weights = Nk/number_of_points

    for ki in range(k):
        means[ki] = (responsibilities[:, ki] @ dataset) / Nk[ki]
        centered_data = dataset - means[ki]
        covariance[ki] = (responsibilities[:, ki, np.newaxis] * centered_data).T @ centered_data / Nk[ki]

    likelihood = np.sum(np.log(np.sum([weights[ki] * multivariate_normal.pdf(dataset, means[ki], covariance[ki]) for ki in range(k)], axis=0)))

    if np.abs(likelihood - prev_likelihood) < threshold:
        print(f"Converged after {iteration + 1} iterations.")
        break
    prev_likelihood = likelihood

print(f"Final means:", means)
print("Final covariances:", covariance)
print("Final weights:", weights)

number of points 6000
dimensions 2
Top 5 entries:
 [[7.57104365 3.53027417]
 [7.33721752 4.26271316]
 [3.07182783 1.11801871]
 [6.22685121 3.66748946]
 [3.51314173 1.60312499]]
Initial means: [[0.03633003 0.39923534]
 [0.74684203 0.85496775]]
Initial covariance: [[[1.36232295 1.14056652]
  [1.14056652 0.9682258 ]]

 [[0.69226379 0.4492413 ]
  [0.4492413  1.13239383]]] (2, 2, 2)
weights [0.82195032 0.17804968]
Converged after 58 iterations.
Final means: [[2.99413012 3.0520968 ]
 [7.01314743 3.98313368]]
Final covariances: [[[1.01023144 0.0271917 ]
  [0.0271917  2.93782429]]

 [[0.97476047 0.49747119]
  [0.49747119 1.00114334]]]
Final weights: [0.33479548 0.66520452]


In [6]:
file_path = os.environ.get("TWO_GAUSSIAN_DATASET_HW2A_QUESTION3")
file_3gaussian = file_path + "3gaussian_hw2A_q3.txt"
dataset_3gaussian = np.loadtxt(file_3gaussian)

number_of_points_3gaussian, dimensions_3gaussian = dataset_3gaussian.shape
k_3gaussian = 3

# means_3gaussian = np.random.choice(number_of_points_3gaussian, size=k_3gaussian, replace=False)
means_3gaussian = np.random.random((3,2))
print("Initial index of random points chosen:", means_3gaussian)
# means_3gaussian = dataset_3gaussian[means_3gaussian]
covariance_3gaussian = np.array([random_cov_matrix() for _ in range(k_3gaussian)])
weights_3gaussian = np.random.dirichlet([2, 4, 7])

print("Initial random means:", means_3gaussian)
print("Initial covariance:", covariance_3gaussian)
print("Initial weights:", weights_3gaussian)

threshold = 1e-8
max_iter = 100
prev_likelihood_3gaussian = -np.inf


for iteration in range(max_iter):

    responsibilities_3gaussian = np.zeros((number_of_points_3gaussian, k_3gaussian))

    for k_i in range(k_3gaussian):
        # responsibilities_3gaussian[:, k_i] = weights_3gaussian[k_i] * multivariate_normal.pdf(dataset_3gaussian, means_3gaussian[k_i], covariance_3gaussian[k_i])
        responsibilities_3gaussian[:, k_i] = weights_3gaussian[k_i] * calculate_multivariate_normal_distribution(dataset_3gaussian, means_3gaussian[k_i], covariance_3gaussian[k_i], k_3gaussian)

    responsibilities_3gaussian /= responsibilities_3gaussian.sum(axis=1, keepdims=1)

    Nk_3gaussian = responsibilities_3gaussian.sum(axis=0)
    weights_3gaussian = Nk_3gaussian/number_of_points_3gaussian

    for k_i in range(k_3gaussian):
        means_3gaussian[k_i] = (responsibilities_3gaussian[:, k_i] @ dataset_3gaussian) / Nk_3gaussian[k_i]
        centered_data_3gaussian = dataset_3gaussian - means_3gaussian[k_i]
        covariance_3gaussian[k_i] = (responsibilities_3gaussian[:, k_i, np.newaxis] * centered_data_3gaussian).T @ centered_data_3gaussian / Nk_3gaussian[k_i]

    likelihood_3gaussian = np.sum(np.log(np.sum([weights_3gaussian[k_i] * multivariate_normal.pdf(dataset_3gaussian, means_3gaussian[k_i], cov=covariance_3gaussian[k_i]) for k_i in range(k_3gaussian)], axis=0)))
    
    if np.abs(likelihood_3gaussian - prev_likelihood_3gaussian) < threshold:
        print(f"Converged after {iteration + 1} iterations.")
        break
    prev_likelihood_3gaussian = likelihood_3gaussian

print(f"Final means:", means_3gaussian)
print("Final covariances:", covariance_3gaussian)
print("Final weights:", weights_3gaussian)

Initial index of random points chosen: [[0.57658723 0.49898512]
 [0.35683543 0.24649119]
 [0.75097232 0.61968612]]
Initial random means: [[0.57658723 0.49898512]
 [0.35683543 0.24649119]
 [0.75097232 0.61968612]]
Initial covariance: [[[0.82946078 0.92622259]
  [0.92622259 1.23891888]]

 [[0.41064337 0.17598749]
  [0.17598749 0.11474408]]

 [[0.08545199 0.11329867]
  [0.11329867 0.3331271 ]]]
Initial weights: [0.1859191  0.34724138 0.46683952]
Converged after 95 iterations.
Final means: [[5.01171924 7.00146402]
 [7.02156035 4.01546007]
 [3.03968325 3.04845984]]
Final covariances: [[[0.97972462 0.18516483]
  [0.18516483 0.97455526]]

 [[0.99041508 0.50096024]
  [0.50096024 0.99564881]]

 [[1.02849525 0.02680653]
  [0.02680653 3.38463545]]]
Final weights: [0.49596938 0.29843675 0.20559387]
